In [9]:
import pygame
import numpy as np

def normalize(vector):
    norm = np.linalg.norm(vector)
    return vector / norm if norm else vector

def limit_speed(vector, max_speed):
    if np.linalg.norm(vector) > max_speed:
        return normalize(vector) * max_speed
    return vector

# Supporting Functions from Pseudo-Code
def get_new_orientation(current_orientation, velocity):
    if np.linalg.norm(velocity) > 0:
        return np.arctan2(velocity[1], velocity[0])
    return current_orientation

def get_cohesion_vector(boid, neighbors):
    if not neighbors:
        return np.array([0, 0])
    center_of_mass = np.mean([neighbor.position for neighbor in neighbors], axis=0)
    return normalize(center_of_mass - boid.position)

def get_separation_vector(boid, neighbors, separation_distance=15):
    move = np.array([0, 0])
    close_neighbors = 0
    for neighbor in neighbors:
        distance = np.linalg.norm(boid.position - neighbor.position)
        if neighbor != boid and distance < separation_distance:
            move -= (neighbor.position - boid.position)
            close_neighbors += 1

    if close_neighbors > 0:
        move /= close_neighbors  # Average out the separation vector

    return normalize(move) if np.any(move) else move


def get_alignment_vector(boid, neighbors):
    if not neighbors:
        return np.array([0, 0])
    average_velocity = np.mean([neighbor.velocity for neighbor in neighbors], axis=0)
    return normalize(average_velocity)

# Player Class
class Player:
    def __init__(self, position, speed, boat_parts_needed):
        self.position = position
        self.speed = speed
        self.has_torch=False
        self.use_torch=False
        self.boat_parts_collected = 0
        self.boat_parts_needed = boat_parts_needed

    def move(self, direction):
        self.position += normalize(direction) * self.speed
#         if(self.position[0])

    def collect_boat_part(self):
        self.boat_parts_collected += 1

    def has_collected_all_parts(self):
        return self.boat_parts_collected >= self.boat_parts_needed

# Enemy Class
class Enemy:
    def __init__(self, position, speed, behavior):
        self.position = position
        self.speed = speed
        self.behavior = behavior  # 'seek', 'flee', or 'flock'
        self.size = (10, 10)  # Width and height of the creature
        self.rect = pygame.Rect(position[0], position[1], self.size[0], self.size[1])

    def update(self, player_position,enemies,obstacles,flock_neighbors=None):
        if self.behavior == 'seek':
            self.position += seek(self.position, player_position, self.speed)
        elif self.behavior == 'flee':
            self.position += flee(self.position, player_position, self.speed, panic_distance=100)
        elif self.behavior == 'flock' and flock_neighbors:
            neighbors = [enemy for enemy in enemies if enemy != self and np.linalg.norm(enemy.position - self.position) < 50]
            cohesion = get_cohesion_vector(self, neighbors)
            separation = get_separation_vector(self, neighbors)
            alignment = get_alignment_vector(self, neighbors)

            flocking_vector = cohesion + separation + alignment
            self.velocity = limit_speed(flocking_vector, max_speed)
            self.position += self.velocity
        self.rect.topleft = self.position
        self.avoid_obstacles(obstacles)
        self.position += self.avoid_overlap(enemies, min_distance=15)

    def avoid_overlap(self, enemies, min_distance):
        move = np.array([0.0, 0.0])
        for other in enemies:
            if other != self:
                distance = np.linalg.norm(self.position - other.position)
                if distance < min_distance:
                    move += (self.position - other.position)
        return normalize(move) if np.linalg.norm(move) > 0 else np.zeros_like(self.position)
    
    def avoid_obstacles(self, obstacles, buffer=10):
        for obstacle in obstacles:
            if obstacle.rect.colliderect(self.rect):
                # Simple obstacle avoidance: move away from the obstacle
                # You may want to make this more sophisticated based on your game's needs
                if self.position[0] < obstacle.rect.x:
                    self.position[0] -= buffer
                else:
                    self.position[0] += buffer
                if self.position[1] < obstacle.rect.y:
                    self.position[1] -= buffer
                else:
                    self.position[1] += buffer
        self.rect.topleft = self.position

class Obstacle:
    def __init__(self, x, y, width, height):
        self.rect = pygame.Rect(x, y, width, height)

    def draw(self, screen):
        pygame.draw.rect(screen, (139, 69, 19), self.rect)  # Brown color for obstacles

# Steering Behaviors
def seek(current_position, target_position, max_speed):
    desired_velocity = normalize(target_position - current_position) * max_speed
    return desired_velocity

def flee(current_position, threat_position, max_speed, panic_distance):
    if np.linalg.norm(current_position - threat_position) < panic_distance:
        return normalize(current_position - threat_position) * max_speed
    return np.zeros_like(current_position)

def flock(neighbors, current_position, max_speed):
    separation_vec = get_separation_vector(boid, neighbors, separation_distance=15)
    alignment_vec = get_alignment_vector(boid, neighbors)
    cohesion_vec = get_cohesion_vector(boid, neighbors)

    # Weights can be adjusted for different behaviors
    separation_weight = 102.5
    alignment_weight = 1.0
    cohesion_weight = 1.0

    flocking_movement = (separation_vec * separation_weight +
                         alignment_vec * alignment_weight +
                         cohesion_vec * cohesion_weight)

    return normalize(flocking_movement) * max_speed

def keep_within_bounds(entity):
    entity.position[0] = np.clip(entity.position[0], 0, 1070)
    entity.position[1] = np.clip(entity.position[1], 0, 670)
    
# Collision Detection
def is_collision(entity, obstacles):
    entity_rect = pygame.Rect(entity.position[0], entity.position[1], 10, 10)  # Assuming size of entity
    return any(entity_rect.colliderect(obstacle.rect) for obstacle in obstacles)

def draw_text(screen, text, size, color, x, y):
    font = pygame.font.Font(None, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    screen.blit(text_surface, text_rect)

# Game Loop
def game_loop():
    player = Player(np.array([550.0, 350.0]), speed=5, boat_parts_needed=3)
    enemies = [Enemy(np.random.rand(2) * 100, speed=0, behavior='seek') for _ in range(5)]
    boat_parts = [np.random.rand(2) * 500 for _ in range(player.boat_parts_needed)]
    escape_point = np.random.rand(2) * 500
    torch_position = np.random.rand(2) * np.array([1100, 700])
    obstacles = [Obstacle(600, 600, 50, 50), Obstacle(650, 508, 100, 50)]
    last_cycle_change = pygame.time.get_ticks()
    is_daytime = True  # Start with day
    while True:
        current_time = pygame.time.get_ticks()
        if (current_time - last_cycle_change) > 10 * 1000:  # Convert to milliseconds
            is_daytime = not is_daytime
            last_cycle_change = current_time
        if is_daytime:
            game.fill((135, 206, 250))
            for enemy in enemies:
                enemy.speed=0.5
        else:
            game.fill((25, 25, 112))
            for enemy in enemies:
                enemy.speed=2
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT: 
                return
            
        original_position = player.position.copy()
        # Update Player
        # Here you should capture user input to set direction
        keys = pygame.key.get_pressed()
        # Player Movement
        player_direction = np.array([0, 0])
        if keys[pygame.K_LEFT] or keys[pygame.K_a]:
            player_direction += np.array([-1, 0])
        if keys[pygame.K_RIGHT] or keys[pygame.K_d]:
            player_direction += np.array([1, 0])
        if keys[pygame.K_UP] or keys[pygame.K_w]:
            player_direction += np.array([0, -1])
        if keys[pygame.K_DOWN] or keys[pygame.K_s]:
            player_direction += np.array([0, 1])
        if keys[pygame.K_f]:
            if(player.has_torch and player.use_torch==False):
                player.use_torch=True
                for enemy in enemies:
                    enemy.behavior='flee'
            elif(player.has_torch and player.use_torch):
                player.use_torch=False
                for enemy in enemies:
                    enemy.behavior='seek'
        if keys[pygame.K_e]:
            # Check for boat part collection
            for part in boat_parts:
                if np.linalg.norm(player.position - part) < 20:  # Assuming a collection radius
                    player.collect_boat_part()
                    boat_parts = np.array([p for p in boat_parts if not np.array_equal(p, part)])
        
            if (torch_position is not None) and (np.linalg.norm(player.position - torch_position) < 15):  # Adjust proximity as needed
                player.has_torch = True
                torch_position = None  # Remove torch from the game
        
        if np.linalg.norm(player_direction) > 0:
            player.move(normalize(player_direction) * 1)
        
        if is_collision(player, obstacles):
            player.position = original_position 
                
        # Update Enemies
        for enemy in enemies:
            enemy.update(player.position,enemies,obstacles)
            if np.linalg.norm(player.position - enemy.position) < 5:  # Player caught
                game.fill((0,0,0))
                draw_text(game, "Game Over! Player caught by enemy. Press any key to exit.", 36, (255, 255, 255), 550, 350)
                pygame.display.flip() 
                waiting_for_input = True
                while waiting_for_input:
                    for event in pygame.event.get():
                        if event.type == pygame.QUIT:
                            waiting_for_input = False
                            running = False
                        elif event.type == pygame.KEYDOWN:
                            # If a key is pressed, exit the blocking loop
                            waiting_for_input = False
                return

        # Check Win Condition
        if player.has_collected_all_parts() and np.linalg.norm(player.position - escape_point) < 10:
            game.fill((0,0,0))
            draw_text(game, "Congratulations! You Escaped the Island. Press any key to exit.", 36, (255, 255, 255), 550, 350)
            pygame.display.flip() 
            waiting_for_input = True
            while waiting_for_input:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        waiting_for_input = False
                        running = False
                    elif event.type == pygame.KEYDOWN:
                        # If a key is pressed, exit the blocking loop
                        waiting_for_input = False
            return

        keep_within_bounds(player)
        for enemy in enemies:
            keep_within_bounds(enemy)
        
        # Rendering
        # In a real game, here you would draw your game frame
#         pygame.draw.circle(game, (0, 200, 0), (int(player.position[0]), int(player.position[1])), 10)
        game.blit(player_image,player.position)  
    
        for enemy in enemies:
#             pygame.draw.circle(game, (255, 0, 0), (int(enemy.position[0]), int(enemy.position[1])), 10)
            game.blit(creature_image,enemy.position)

        for part in boat_parts:
#             pygame.draw.circle(game, (255, 145, 0), (int(part[0]), int(part[1])), 5)    
            game.blit(boat_image,part)

#         pygame.draw.rect(game, (255, 255, 255), (int(escape_point[0]), int(escape_point[1]), 20, 20))
        game.blit(door_image,escape_point)

        if torch_position is not None:
#             pygame.draw.circle(game, (255, 255, 0), (int(torch_position[0]), int(torch_position[1])), 10)  # Draw torch
            game.blit(torch_image,torch_position)

        for obstacle in obstacles:
            obstacle.draw(game)
        pygame.display.flip() 
        
pygame.init()
boat_image = pygame.image.load('images/boat.jpeg')
torch_image = pygame.image.load('images/torch.jpg')
player_image = pygame.image.load('images/player.jpeg')
creature_image = pygame.image.load('images/monster.jpeg')
door_image = pygame.image.load('images/door.jpeg')
game=pygame.display.set_mode((1100,700))
pygame.display.set_caption("Island Escape")    
# Run the game
game_loop()
pygame.quit()

In [16]:
import pygame
import numpy as np

def normalize(vector):
    norm = np.linalg.norm(vector)
    return vector / norm if norm else vector

def limit_speed(vector, max_speed):
    if np.linalg.norm(vector) > max_speed:
        return normalize(vector) * max_speed
    return vector

# Supporting Functions from Pseudo-Code
def get_new_orientation(current_orientation, velocity):
    if np.linalg.norm(velocity) > 0:
        return np.arctan2(velocity[1], velocity[0])
    return current_orientation

def get_cohesion_vector(boid, neighbors):
    if not neighbors:
        return np.array([0, 0])
    center_of_mass = np.mean([neighbor.position for neighbor in neighbors], axis=0)
    return normalize(center_of_mass - boid.position)

def get_separation_vector(boid, neighbors, separation_distance=15):
    move = np.array([0, 0])
    for neighbor in neighbors:
        if neighbor != boid and np.linalg.norm(boid.position - neighbor.position) < separation_distance:
            move -= (neighbor.position - boid.position)
    return normalize(move) if np.any(move) else move

def get_alignment_vector(boid, neighbors):
    if not neighbors:
        return np.array([0, 0])
    average_velocity = np.mean([neighbor.velocity for neighbor in neighbors], axis=0)
    return normalize(average_velocity)

# Player Class
class Player:
    def __init__(self, position, speed, boat_parts_needed):
        self.position = position
        self.speed = speed
        self.has_torch=False
        self.use_torch=False
        self.boat_parts_collected = 0
        self.boat_parts_needed = boat_parts_needed

    def move(self, direction):
        self.position += normalize(direction) * self.speed
#         if(self.position[0])

    def collect_boat_part(self):
        self.boat_parts_collected += 1

    def has_collected_all_parts(self):
        return self.boat_parts_collected >= self.boat_parts_needed

# Enemy Class
class Enemy:
    def __init__(self, position, speed, behavior):
        self.position = position
        self.speed = speed
        self.behavior = behavior  # 'seek', 'flee', or 'flock'

    def update(self, player_position, flock_neighbors=None):
        if self.behavior == 'seek':
            self.position += seek(self.position, player_position, self.speed)
        elif self.behavior == 'flee':
            self.position += flee(self.position, player_position, self.speed, panic_distance=100)
        elif self.behavior == 'flock' and flock_neighbors:
            neighbors = [enemy for enemy in enemies if enemy != self and np.linalg.norm(enemy.position - self.position) < 50]
            cohesion = get_cohesion_vector(self, neighbors)
            separation = get_separation_vector(self, neighbors)
            alignment = get_alignment_vector(self, neighbors)

            flocking_vector = cohesion + separation + alignment
            self.velocity = limit_speed(flocking_vector, max_speed)
            self.position += self.velocity

class Obstacle:
    def __init__(self, x, y, width, height):
        self.rect = pygame.Rect(x, y, width, height)

    def draw(self, screen):
        pygame.draw.rect(screen, (139, 69, 19), self.rect)  # Brown color for obstacles

# Steering Behaviors
def seek(current_position, target_position, max_speed):
    desired_velocity = normalize(target_position - current_position) * max_speed
    return desired_velocity

def flee(current_position, threat_position, max_speed, panic_distance):
    if np.linalg.norm(current_position - threat_position) < panic_distance:
        return normalize(current_position - threat_position) * max_speed
    return np.zeros_like(current_position)

def flock(neighbors, current_position, max_speed):
    separation_vec = get_separation_vector(boid, neighbors, separation_distance=15)
    alignment_vec = get_alignment_vector(boid, neighbors)
    cohesion_vec = get_cohesion_vector(boid, neighbors)

    # Weights can be adjusted for different behaviors
    separation_weight = 1.5
    alignment_weight = 1.0
    cohesion_weight = 1.0

    flocking_movement = (separation_vec * separation_weight +
                         alignment_vec * alignment_weight +
                         cohesion_vec * cohesion_weight)

    return normalize(flocking_movement) * max_speed

def keep_within_bounds(entity):
    entity.position[0] = np.clip(entity.position[0], 0, 1070)
    entity.position[1] = np.clip(entity.position[1], 0, 670)
    
# Collision Detection
def is_collision(entity, obstacles):
    entity_rect = pygame.Rect(entity.position[0], entity.position[1], 10, 10)  # Assuming size of entity
    return any(entity_rect.colliderect(obstacle.rect) for obstacle in obstacles)

def draw_text(screen, text, size, color, x, y):
    font = pygame.font.Font(None, size)
    text_surface = font.render(text, True, color)
    text_rect = text_surface.get_rect()
    text_rect.midtop = (x, y)
    screen.blit(text_surface, text_rect)

# Game Loop
def game_loop():
    player = Player(np.array([550.0, 350.0]), speed=5, boat_parts_needed=3)
    enemies = [Enemy(np.random.rand(2) * 100, speed=0, behavior='seek') for _ in range(5)]
    boat_parts = [np.random.rand(2) * 500 for _ in range(player.boat_parts_needed)]
    escape_point = np.random.rand(2) * 500
    torch_position = np.random.rand(2) * np.array([1100, 700])
    obstacles = [Obstacle(600, 600, 50, 50), Obstacle(650, 508, 100, 50)]
    last_cycle_change = pygame.time.get_ticks()
    is_daytime = True  # Start with day
    while True:
        current_time = pygame.time.get_ticks()
        if (current_time - last_cycle_change) > 10 * 1000:  # Convert to milliseconds
            is_daytime = not is_daytime
            last_cycle_change = current_time
        if is_daytime:
            game.fill((135, 206, 250))
            for enemy in enemies:
                enemy.speed=0.5
        else:
            game.fill((25, 25, 112))
            for enemy in enemies:
                enemy.speed=2
        for event in pygame.event.get(): 
            if event.type == pygame.QUIT: 
                return
            
        original_position = player.position.copy()
        # Update Player
        # Here you should capture user input to set direction
        keys = pygame.key.get_pressed()
        # Player Movement
        player_direction = np.array([0, 0])
        if keys[pygame.K_LEFT] or keys[pygame.K_a]:
            player_direction += np.array([-1, 0])
        if keys[pygame.K_RIGHT] or keys[pygame.K_d]:
            player_direction += np.array([1, 0])
        if keys[pygame.K_UP] or keys[pygame.K_w]:
            player_direction += np.array([0, -1])
        if keys[pygame.K_DOWN] or keys[pygame.K_s]:
            player_direction += np.array([0, 1])
        if keys[pygame.K_f]:
            if(player.has_torch and player.use_torch==False):
                player.use_torch=True
                for enemy in enemies:
                    enemy.behavior='flee'
            elif(player.has_torch and player.use_torch):
                player.use_torch=False
                for enemy in enemies:
                    enemy.behavior='seek'
        if keys[pygame.K_e]:
            # Check for boat part collection
            for part in list(boat_parts):  # Iterate over a copy of the list to safely modify the original
                if np.linalg.norm(player.position - part) < 20:  # Assuming a collection radius
                    player.collect_boat_part()
                    boat_parts.remove(part)  # Remove the collected part from the list

        
            if (torch_position is not None) and (np.linalg.norm(player.position - torch_position) < 15):  # Adjust proximity as needed
                player.has_torch = True
                torch_position = None  # Remove torch from the game
        
        if np.linalg.norm(player_direction) > 0:
            player.move(normalize(player_direction) * 1)
        
        if is_collision(player, obstacles):
            player.position = original_position 
                
        # Update Enemies
        for enemy in enemies:
            original_pos=enemy.position
            enemy.update(player.position)
            if is_collision(enemy,obstacles):
                enemy.position=original_pos
            if np.linalg.norm(player.position - enemy.position) < 5:  # Player caught
                game.fill((0,0,0))
                draw_text(game, "Game Over! Player caught by enemy. Press any key to exit.", 36, (255, 255, 255), 550, 350)
                pygame.display.flip() 
                waiting_for_input = True
                while waiting_for_input:
                    for event in pygame.event.get():
                        if event.type == pygame.QUIT:
                            waiting_for_input = False
                            running = False
                        elif event.type == pygame.KEYDOWN:
                            # If a key is pressed, exit the blocking loop
                            waiting_for_input = False
                return

        # Check Win Condition
        if player.has_collected_all_parts() and np.linalg.norm(player.position - escape_point) < 10:
            game.fill((0,0,0))
            draw_text(game, "Congratulations! You Escaped the Island. Press any key to exit.", 36, (255, 255, 255), 550, 350)
            pygame.display.flip() 
            waiting_for_input = True
            while waiting_for_input:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        waiting_for_input = False
                        running = False
                    elif event.type == pygame.KEYDOWN:
                        # If a key is pressed, exit the blocking loop
                        waiting_for_input = False
            return

        keep_within_bounds(player)
        for enemy in enemies:
            keep_within_bounds(enemy)
        
        # Rendering
        # In a real game, here you would draw your game frame
#         pygame.draw.circle(game, (0, 200, 0), (int(player.position[0]), int(player.position[1])), 10)
        game.blit(player_image,player.position)  
    
        for enemy in enemies:
#             pygame.draw.circle(game, (255, 0, 0), (int(enemy.position[0]), int(enemy.position[1])), 10)
            game.blit(creature_image,enemy.position)

        for part in boat_parts:
#             pygame.draw.circle(game, (255, 145, 0), (int(part[0]), int(part[1])), 5)    
            game.blit(boat_image,part)

#         pygame.draw.rect(game, (255, 255, 255), (int(escape_point[0]), int(escape_point[1]), 20, 20))
        game.blit(door_image,escape_point)

        if torch_position is not None:
#             pygame.draw.circle(game, (255, 255, 0), (int(torch_position[0]), int(torch_position[1])), 10)  # Draw torch
            game.blit(torch_image,torch_position)

        for obstacle in obstacles:
            obstacle.draw(game)
        pygame.display.flip() 
        
pygame.init()
boat_image = pygame.image.load('images/boat.jpeg')
torch_image = pygame.image.load('images/torch.jpg')
player_image = pygame.image.load('images/player.jpeg')
creature_image = pygame.image.load('images/monster.jpeg')
door_image = pygame.image.load('images/door.jpeg')
game=pygame.display.set_mode((1100,700))
pygame.display.set_caption("Island Escape")    
# Run the game
game_loop()
pygame.quit()